# Caso de Estudio: Riesgo Crediticio

*DESCARGO DE RESPONSABILIDAD: esta es un planteamiento para problemas de clasificación. El texto que se documenta proviene de diferentes fuentes (descripción del conjunto de datos de kaggle, documentación de sklearn, documentación de matplotlib, wikipedia, etc.), para sustento del proyecto FEDU de la Universidad Nacional San Antonio Abad del Cusco.*

## Table of content

* [Descripción general del conjunto de datos](#ds)
* [Análisis exploratorio](#explo)
    * [Estadísticas descriptivas de préstamos PAGADOS](#descp)
    * [Estadísticas descriptivas para préstamos INCUMPLIDOS](#descd)
    * [INCUMPLIMIENTO en función del motivo de adquisición de los préstamos](#reason)
    * [INCUMPLIMIENTO en función de la ocupación](#occupation)
    * [Resumen gráfico](#graph)
    * [Trama de violín](#violin)
    * [Matriz de correlación](#corr)
* [Prueba de clasificadores predeterminados](#classification)
* [Evaluación del modelo](#eval)
    * [Precisión y recuperación](#per)
    * [F1](#f1)
    * [Característica Operativa del Receptor](#roc)
    * [Matriz de confusión](#confusion)
    * [robabilidad de clasificación](#prob)
* [Regresión logística](#logit)
* [Clasificador SGD](#sgd)
* [Clasificador de vectores de apoyo](#svc)
* [Clasificador de aumento de gradiente](#gbrt)
* [Bosque de árbol al azar](#frt)
    * [Clasificador de bosque aleatorio](#rfc)
    * [Árbol extremadamente aleatorio](#ert)
* [Comparación y conclusión del modelo](#conclusion)

## Visión General del Dataset
<a id='ds'></a>

El conjunto de datos utilizado contiene información de referencia y de rendimiento de préstamos para 5,960 préstamos. El objetivo (BAD) es una variable binaria que indica si un solicitante finalmente incurrió en incumplimiento o en grave mora en alguna entidad bancaria. Y por muestreo rápido. Este resultado adverso se produjo en 1.189 casos (20%) del total de la muestra.

Para cada aspirante a una tarjeta de crédito, se registraron 11 variables de entrada:

> * BAD: 1 = candidato con préstamo incumplido o con mora; 0 = candidato que paga su deuda y no tiene registro negativo
* LOAN: Monto de solicitud de préstamo
* MORTDUE: Monto adeudado de la hipoteca existente
* VALUE: Valor actual del bien o propiedad
* REASON: DebtCon = consolidación de la deuda; HomeImp = mejoras para el hogar
* JOB: Categorias ocupacionales o profesionales
* YOJ: Años es esu trabajo actual
* DEROG: Número de informes derogados o cancelados importantes
* DELINQ: Número de lineas de crédito morosas
* CLAGE: Antiguedad de la linea de crédito más antigua en meses
* NINQ:Número de consultas crediticas recientes
* CLNO: Número de líneas de crédito

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

import holoviews as hv
hv.extension('bokeh', 'matplotlib', logo=False)

# Evitar que aparezcan advertencias
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('../input/hmeq.csv', low_memory=False) # Sin columnas duplicadas, sin columnas altamente correlacionadas
df.drop('DEBTINC', axis=1, inplace=True) # El significado de esta variable no está claro. Por lo que se excluye del planteamiento
df.dropna(axis=0, how='any', inplace=True)

## Análisis Exploratorio
<a id='explo'></a>

Se resume las principales características del conjunto de datos con métodos visuales y estadísticas resumidas. Se utiliza la variable objetivo (BAD) para dividir el conjunto de datos en submuestras y se busca específicamente variables, características y correlación que contengan poder de clasificación.

### Estadísticas descriptivas de préstamos PAGADOS

<a id='descp'></a>

In [ ]:
df[df['BAD']==0].drop('BAD', axis=1).describe().style.format("{:.2f}")

### Estadísticas descriptivas para préstamos INCUMPLIDOS
<a id='descd'></a>

In [ ]:
df[df['BAD']==1].drop('BAD', axis=1).describe().style.format("{:.2f}")

1. De las estadísticas descriptivas anteriores, podemos sacar la siguiente consideración:

* El monto del préstamo solicitado, el monto de la hipoteca adeudada y el valor de la garantía subyacente son estadísticamente consistentes tanto para los préstamos que fueron PAGADOS como para los que resultaron en INCUMPLIMIENTO. Esto sugiere que esas variables pueden no proporcionar un poder de discriminación significativo para separar las dos clases.

* El número de años en el trabajo actual (YOJ) parece discriminar las dos clases, ya que los INCUMPLIMIENTOS parecen más frecuentes en los contratistas que tienen una antigüedad más corta. Esta tendencia está respaldada por los cuantiles de valor promedio correspondientes, que indican una distribución sesgada hacia una antigüedad más corta.

* Se aplican consideraciones similares a las variables relacionadas con el historial crediticio del contratista, tales como: el número de informes despectivos importantes (DEROG), el número de líneas de crédito morosas (DELINQ), la antigüedad de la línea de crédito más antigua en meses (CLAGE) y el número de consultas crediticias recientes (NINQ). En el caso de INCUMPLIMIENTO, la distribución de estas variables está sesgada hacia valores que sugieren una historia crediticia peor que la distribución correspondiente para los contratistas de préstamos PAGADOS.

* Finalmente, el número de líneas de crédito abiertas (CLNO) parece estadísticamente consistente en ambos casos, lo que sugiere que esta variable no tiene un poder de discriminación significativo.

In [ ]:
df.loc[df.BAD == 1, 'STATUS'] = 'DEFAULT'
df.loc[df.BAD == 0, 'STATUS'] = 'PAID'

### INCUMPLIMIENTO en función del motivo de adquisición de los préstamos
<a id='reason'></a>
La fracción de préstamos PAGADOS e INCUMPLIDOS no parece depender en gran medida del motivo de la adquisición del préstamo. En promedio, se ha pagado el 80% de los préstamos, mientras que el 20% está en INCUMPLIMIENTO. La discrepancia del 2% observada no es estadísticamente significativa dada la cantidad de préstamos en el conjunto de datos.


In [ ]:
g = df.groupby('REASON')
g['STATUS'].value_counts(normalize=True).to_frame().style.format("{:.1%}")

* ###  INCUMPLIMIENTO en función de la ocupación

<a id='occupation'></a>
La fracción de préstamos PAGADOS e INCUMPLIDOS muestra cierta dependencia de la ocupación del contratista. Los trabajadores de oficina y los ejecutivos profesionales tienen la mayor probabilidad de pagar sus préstamos, mientras que los vendedores y los autónomos tienen la mayor probabilidad de impago. La ocupación muestra un buen poder de discriminación y probablemente será una característica importante de nuestro modelo de clasificación.


In [ ]:
g = df.groupby('JOB')
g['STATUS'].value_counts(normalize=True).to_frame().style.format("{:.1%}")

In [ ]:
%%opts Bars[width=700 height=400 tools=['hover'] xrotation=45]{+axiswise +framewise}

# Categorical

cols = ['REASON', 'JOB']

dd={}

for col in cols:

    counts=df.groupby(col)['STATUS'].value_counts(normalize=True).to_frame('val').reset_index()
    dd[col] = hv.Bars(counts, [col, 'STATUS'], 'val') 
    
var = [*dd]
kdims=hv.Dimension(('var', 'Variable'), values=var)    
hv.HoloMap(dd, kdims=kdims)

### Resumen gráfico

<a id='graph'></a>
A continuación se muestra una descripción gráfica coherente del conjunto de datos. Para cada variable, muestro un histograma para todo el conjunto de datos, para los préstamos PAGADOS y DEFUALT, respectivamente. Las correlaciones entre variables también se resumen en diagramas de dispersión bidimensionales.


In [ ]:
%%opts Histogram[width=700 height=400 tools=['hover'] xrotation=0]{+axiswise +framewise}

g = df.groupby('STATUS')

cols = ['LOAN',
        'MORTDUE', 
        'VALUE',
        'YOJ',
        'DEROG',
        'DELINQ',
        'CLAGE',
        'NINQ',
        'CLNO']
dd={}

# Histograms
for col in cols:
    
    freq, edges = np.histogram(df[col].values)
    dd[col] = hv.Histogram((edges, freq), label='TODOS los préstamos').redim.label(x=' ')
    
    freq, edges = np.histogram(g.get_group('PAID')[col].values, bins=edges)
    dd[col] *= hv.Histogram((edges, freq), label='Préstamos PAGADOS').redim.label(x=' ')
    
    freq, edges = np.histogram(g.get_group('DEFAULT')[col].values, bins=edges)
    dd[col] *= hv.Histogram((edges, freq), label='Préstamos INCUMPLIDOS' ).redim.label(x=' ')   
    
var = [*dd]
kdims=hv.Dimension(('var', 'Variable'), values=var)    
hv.HoloMap(dd, kdims=kdims)

In [ ]:
%%opts Scatter[width=500 height=500 tools=['hover'] xrotation=0]{+axiswise +framewise}

g = df.groupby('STATUS')

cols = ['LOAN',
        'MORTDUE',
        'VALUE',
        'YOJ',
        'DEROG',
        'DELINQ',
        'CLAGE',
        'NINQ',
        'CLNO']

import itertools
prod = list(itertools.combinations(cols,2))

dd = {}

for p in prod:
    dd['_'.join(p)] = hv.Scatter(g.get_group('PAID')[list(p)], label='Préstamos PAGADOS').options(size=5)
    dd['_'.join(p)] *= hv.Scatter(g.get_group('DEFAULT')[list(p)], label='Préstamos INCUMPLIDOS').options(size=5, marker='x')
    
var = [*dd]
kdims=hv.Dimension(('var', 'Variable'), values=var)    
hv.HoloMap(dd, kdims=kdims).collate()

In [ ]:
g=sns.PairGrid(df.drop('BAD',axis=1), hue='STATUS', diag_sharey=False, palette={'PAID': 'C0', 'DEFAULT':'C1'})
g.map_lower(sns.kdeplot)
g.map_upper(sns.scatterplot)
g.map_diag(sns.kdeplot, lw=3)
g.add_legend()
plt.show()

### Trama de violín
<a id='violin'></a>
La gráfica de violín muestra las diferentes formas de la función de densidad de probabilidad para algunas de las variables discutidas anteriormente que parecen las más prometedoras para la tarea de clasificación. El gráfico muestra, en diferentes colores, los préstamos PAGADOS e INCUMPLIDOS. Las líneas discontinuas horizontales indican la posición de la media y los cuantiles de las diferentes distribuciones. Dado que existe una dependencia de la probabilidad PREDETERMINADA de las categorías de ocupación, se muestran los "violines" para cada una de ellas.

In [ ]:
cols=['YOJ', 'CLAGE', 'NINQ']

for col in cols:
    
    plt.figure(figsize=(15,5))

    sns.violinplot(x='JOB', y=col, hue='STATUS',
                   split=True, inner="quart",  palette={'PAID': 'C0', 'DEFAULT':'C1'},
                   data=df)
    
    sns.despine(left=True)

### Matriz de correlación

<a id='corr'></a>
Finalmente se muestra la matriz de correlación entre las variables discutidas hasta ahora. Las correlaciones son útiles porque pueden indicar una relación predictiva que se puede aprovechar en la tarea de clasificación.

El gráfico está codificado por colores: los colores más fríos corresponden a una baja correlación, mientras que los colores más cálidos corresponden a una alta correlación. Las variables también se agrupan según su correlación, es decir, las variables con mayor correlación están próximas entre sí.

Las variables relacionadas con el historial crediticio (DELINQ, DEROG, NINQ) son las más correlacionadas con el estado del préstamo (BAD), lo que sugiere que estas serán las variables más discriminatorias. Estas variables también están levemente correlacionadas entre ellas, lo que sugiere que parte de la información podría ser redundante.

Como ya se mencionó, el monto del préstamo o la garantía subyacente no parecen estar relacionados con el estado del préstamo. De todos modos, forman otro grupo de correlación con otras variables como la antigüedad de la línea de crédito más antigua (CLAGE) y el número de líneas de crédito (CLNO). Esto es de esperar ya que esas variables están claramente relacionadas.

In [ ]:
def compute_corr(df,size=10):
    '''La función traza una matriz de correlación gráfica para cada par de columnas en el marco de datos.

    Entradas:
        df: pandas DataFrame
        size: vertical y horizontal tamaño del plot'''
    import scipy
    import scipy.cluster.hierarchy as sch
    
    corr = df.corr()
    
    # Clustering
    d = sch.distance.pdist(corr)   # vector of ('55' choose 2) pairwise distances
    L = sch.linkage(d, method='complete')
    ind = sch.fcluster(L, 0.5*d.max(), 'distance')
    columns = [df.select_dtypes(include=[np.number]).columns.tolist()[i] for i in list((np.argsort(ind)))]
    
    # Reordered df upon custering results
    df = df.reindex(columns, axis=1)
    
    # Recompute correlation matrix w/ clustering
    corr = df.corr()
    #corr.dropna(axis=0, how='all', inplace=True)
    #corr.dropna(axis=1, how='all', inplace=True)
    #corr.fillna(0, inplace=True)
    
    #fig, ax = plt.subplots(figsize=(size, size))
    #img = ax.matshow(corr)
    #plt.xticks(range(len(corr.columns)), corr.columns, rotation=45);
    #plt.yticks(range(len(corr.columns)), corr.columns);
    #fig.colorbar(img)
    
    return corr

In [ ]:
%%opts HeatMap [tools=['hover'] colorbar=True width=500  height=500 toolbar='above', xrotation=45, yrotation=45]

corr=compute_corr(df)
corr=corr.stack(level=0).to_frame('value').reset_index()
hv.HeatMap(corr).options(cmap='Viridis')

<a id='classification'></a>
# Prueba de clasificadores predeterminados
El análisis exploratorio descrito anteriormente proporciona una buena perspectiva sobre el conjunto de datos y destaca las variables más prometedoras con un buen poder de discriminación para identificar los préstamos que resultan en INCUMPLIMIENTO. En esta sección, desarrollo e investigo clasificadores de aprendizaje automático superpuestos para predecir el resultado de los préstamos. Dada la gran cantidad de algoritmos disponibles en la literatura, empiezo con los métodos simples, como la regresión logística, y gradualmente aumento la complejidad del modelo hasta las técnicas de árboles aleatorios. Finalmente comparo el desempeño de cada modelo y analizo el más apropiado para esta tarea de clasificación de préstamos. En esta sección se desarrollan los siguientes modelos:

* [Regresión logística](#logit)
* [Clasificador SGD](#sgd)
* [Clasificador de vectores de apoyo](#svc)
* [Clasificador de aumento de gradiente](#gbrt)
* [Bosque de árbol al azar](#frt)
    * [Clasificador de bosque aleatorio](#rfc)
    * [Árbol extremadamente aleatorio](#ert)
* [Comparación y conclusión del modelo](#conclusion)

<a id='eval'></a>
## Evaluación del modelo
La evaluación del desempeño de los clasificadores es relativamente compleja y depende de muchos factores, algunos de los cuales dependen del modelo. Con el fin de identificar el mejor modelo para nuestra tarea de clasificación, adopto diferentes métricas de evaluación que se resumen brevemente a continuación.

Para evitar el sobreentrenamiento, el rendimiento de nuestro modelo de clasificación se evalúa mediante validación cruzada. El conjunto de entrenamiento se divide aleatoriamente en $ N $ subconjuntos distintos llamados pliegues, luego el modelo se entrena y evalúa $ N $ veces mediante el uso de un pliegue diferente para la evaluación de un modelo que se entrena en los otros $ N-1 $ pliegues. Los resultados del procedimiento consisten en puntajes de evaluación de $ N $ para cada métrica que luego se promedian. Estos promedios se utilizan finalmente para comparar las diferentes técnicas consideradas en este estudio.


<a id='per'></a>
### Precisión y recuperación
Precision-Recall es una métrica de rendimiento útil para evaluar un modelo en aquellos casos en que las clases están muy desequilibradas. En la recuperación de información, la precisión es una medida de la relevancia de los resultados, mientras que la recuperación es una medida de cuántos resultados verdaderamente relevantes se devuelven. Intuitivamente, la precisión es la capacidad del clasificador de no etiquetar como positiva una muestra negativa, y la recuperación es la capacidad del clasificador de encontrar todas las muestras positivas.

Un sistema con alta recuperación pero baja precisión devuelve muchas etiquetas que tienden a predecirse incorrectamente en comparación con las etiquetas de entrenamiento. Un sistema con alta precisión pero poca recuperación es todo lo contrario, arrojando muy pocos resultados, pero la mayoría de las etiquetas predichas son correctas en comparación con las etiquetas de entrenamiento. Un sistema ideal con alta precisión y alta recuperación devolverá muchos resultados, con muchos resultados etiquetados correctamente.

La precisión ($ P $) se define como el número de verdaderos positivos ($ T_ {p} $) sobre el número de verdaderos positivos más el número de falsos positivos ($ T_ {p} + F_ {p} $):

$P = \frac{T_{p}}{T_{p}+F_{p}}$  

La recuperación ($ R $) se define como el número de verdaderos positivos ($ T_ {p} $) sobre el número de verdaderos positivos más el número de falsos negativos ($ T_ {p} + F_ {n} $):

$R = \frac{T_{p}}{T_{p}+F_{n}}$

<a id='f1'></a>
### Medida F1
A menudo es conveniente combinar la precisión y la recuperación en una única métrica llamada puntuación $ F_ {1} $, definida como una media armónica ponderada de la precisión y la recuperación:

$F_{1} = 2\times \frac{P \times R}{P+R}$

Mientras que la media regular trata todos los valores por igual, la media armónica da mucho más peso a los valores bajos. Como resultado, el clasificador solo obtendrá una puntuación F1 alta si tanto la memoria como la precisión son altas.
La puntuación $ F_ {1} $ favorece a los clasificadores que tienen una precisión y una recuperación similares. Esto no siempre es lo que desea: en algunos contextos, lo que más le importa es la precisión, y en otros contextos, realmente le importa el recuerdo.

<a id='roc'></a>
###  Característica Operativa del Receptor
Una característica operativa del receptor (ROC), o simplemente una curva ROC, es un gráfico que ilustra el rendimiento de un sistema clasificador binario a medida que varía su umbral de discriminación. Se crea trazando la fracción de verdaderos positivos de los positivos (TPR = tasa de verdaderos positivos) frente a la fracción de falsos positivos de los negativos (FPR = tasa de falsos positivos), en varios valores de umbral. TPR también se conoce como sensibilidad, y FPR es uno menos la especificidad o tasa negativa verdadera.
Hay una compensación: cuanto mayor es la recuperación (TPR), más falsos positivos (FPR) produce el clasificador. La línea de puntos representa la curva ROC de un clasificador puramente aleatorio; un buen clasificador permanece lo más lejos posible de esa línea (hacia la esquina superior izquierda).

El área bajo la curva ROC, que también se denota por AUC, resume la información de la curva en un número. El AUC debe interpretarse como la probabilidad de que un clasificador clasifique una istancia positiva elegida al azar más alta que una negativa elegida al azar. Un clasificador perfecto tendrá un ROC AUC igual a 1, mientras que un clasificador puramente aleatorio tendrá un ROC AUC igual a 0,5.

<a id='confusion'></a>
### Matriz de confusión
La matriz de confusión evalúa la precisión de la clasificación calculando la matriz de confusión con cada fila correspondiente a la clase verdadera. Por definición, la entrada $ i, j $ en una matriz de confusión es el número de observaciones en realidad en el grupo $ i $, pero se predice que estarán en el grupo $ j $. La matriz de confusión no se utiliza para la evaluación del modelo, pero proporciona una buena comprensión del rendimiento general del modelo.

<a id='prob'></a>
### Probabilidad de clasificación
La probabilidad de clasificación proporciona una estimación de la probabilidad de que una instancia determinada de los datos pertenezca a una clase determinada. En un problema de clasificación binaria como el que se está considerando, el histograma de la probabilidad de clasificación para las dos clases proporciona una buena comprensión visual del rendimiento del modelo. Cuanto más alejados estén los picos de la probabilidad de clasificación, mayor será el poder de separación del modelo.

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

In [ ]:
df=pd.read_csv('../input/hmeq.csv', low_memory=False) # Sin columnas duplicadas, sin columnas altamente correlacionadas
df=pd.get_dummies(df, columns=['REASON','JOB'])
df.drop('DEBTINC', axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
y = df['BAD']
X = df.drop(['BAD'], axis=1)

In [ ]:
def cross_validate_model(model, X, y, 
                         scoring=['f1', 'precision', 'recall', 'roc_auc'], 
                         cv=12, n_jobs=-1, verbose=True):
    
    scores = cross_validate(pipe, 
                        X, y, 
                        scoring=scoring,
                        cv=cv, n_jobs=n_jobs, 
                        verbose=verbose,
                        return_train_score=False)

    #sorted(scores.keys())
    dd={}
    
    for key, val in scores.items():
        if key in ['fit_time', 'score_time']:
            continue
        #print('{:>30}: {:>6.5f} +/- {:.5f}'.format(key, np.mean(val), np.std(val)) )
        name = " ".join(key.split('_')[1:]).capitalize()
        
        dd[name] = {'value' : np.mean(val), 'error' : np.std(val)}
        
    return  pd.DataFrame(dd)    
    #print()
    #pprint(scores)
    #print()

In [ ]:
def plot_roc(model, X_test ,y_test, n_classes=0):
    
    from sklearn.metrics import roc_curve, auc
    
    """
    Puntuaciones objetivo, pueden ser estimaciones de probabilidad
    de la clase positiva, valores de confianza o
    medida de decisiones sin umbral (como se devuelve
    por "decision_function" en algunos clasificadores).
    """
    try:
        y_score = model.decision_function(X_test)
    except Exception as e:
        y_score = model.predict_proba(X_test)[:,1]
    
    
    fpr, tpr, _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc = auc(fpr, tpr)

    # Compute micro-average ROC curve and ROC area
    #fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    #roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    #plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de falsos positivos')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.title('Ejemplo de característica de funcionamiento del receptor')
    plt.legend(loc="lower right")
    #plt.show()
    
# mezclar y dividir conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
#                                                    random_state=0)

In [ ]:
def plot_confusion_matrix(model, X_test ,y_test,
                          classes=[0,1],
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    import itertools
    from sklearn.metrics import confusion_matrix
    
    y_pred = model.predict(X_test)
    
    # Calcular matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    
    """
    Esta función imprime y traza la matriz de confusión.
    La normalización se puede aplicar configurando `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #    print("Normalized confusion matrix")
    #else:
    #    print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def feature_importance(coef, names, verbose=False, plot=True):
    
    #importances = model.feature_importances_

    
    
    #std = np.std([tree.feature_importances_ for tree in model.estimators_],
    #             axis=0)
    indices = np.argsort(coef)[::-1]
    
    if verbose:
    
        # Imprime la clasificación de funciones
        print("Feature ranking:")
    
        for f in range(len(names)):
            print("{:>2d}. {:>15}: {:.5f}".format(f + 1, names[indices[f]], coef[indices[f]]))
        
    if plot:
        
        # Trazar la importancia de las características del bosque
        #plt.figure(figsize=(5,10))
        plt.title("Feature importances")
        plt.barh(range(len(names)), coef[indices][::-1], align="center")
        #plt.barh(range(X.shape[1]), importances[indices][::-1],
        #         xerr=std[indices][::-1], align="center")
        plt.yticks(range(len(names)), names[indices][::-1])
        #plt.xlim([-0.001, 1.1])
        #plt.show()

In [ ]:
def plot_proba(model, X, y, bins=40, show_class = 1):
    
    from sklearn.calibration import CalibratedClassifierCV
    
    model = CalibratedClassifierCV(model)#, cv='prefit')
    
    model.fit(X, y)
    
    proba=model.predict_proba(X)
    
    if show_class == 0:
        sns.kdeplot(proba[y==0,0], shade=True, color="r", label='True class')
        sns.kdeplot(proba[y==0,1], shade=True, color="b", label='Wrong class')
        plt.title('Classification probability: Class 0')
    elif show_class == 1:
        sns.kdeplot(proba[y==1,1], shade=True, color="r", label='True class')
        sns.kdeplot(proba[y==1,0], shade=True, color="b", label='Wrong class')
        plt.title('Classification probability: Class 1')
    plt.legend()

## Regresión logística
<a id='logit'></a>

La regresión logística es el modelo lineal más simple para la clasificación. La regresión logística también se conoce en la literatura como regresión logit, clasificación de máxima entropía (MaxEnt) o clasificador log-lineal. En este modelo, las probabilidades que describen los posibles resultados de un solo ensayo se modelan utilizando una función logística. El problema de optimización se resuelve minimizando una función de costo utilizando un algoritmo de descenso de coordenadas altamente optimizado.


In [ ]:
from sklearn.linear_model import LogisticRegression

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', LogisticRegression(random_state=0))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].coef_[0], X.columns)

plt.tight_layout()

In [ ]:
logit_xval_res = cross_validate_model(pipe, X, y, verbose=False)
logit_xval_res.T[['value','error']].style.format("{:.2f}")

<a id='sgd'></a>
## Clasificador SGD
Este algoritmo implementa modelos lineales regularizados con aprendizaje de descenso de gradiente estocástico (SGD): el gradiente de la pérdida se estima en cada muestra a la vez y el modelo se actualiza a lo largo del camino con un programa de fuerza decreciente (también conocido como tasa de aprendizaje). SGD permite el aprendizaje minibatch (en línea / fuera del núcleo).


In [ ]:
from sklearn.linear_model import SGDClassifier

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3, random_state=0))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].coef_[0], X.columns)

plt.tight_layout()

In [ ]:
sgd_xval_res = cross_validate_model(pipe, X, y, verbose=False)
sgd_xval_res.T[['value','error']].style.format("{:.2f}")

## Clasificador de vectores de apoyo

<a id='svc'></a>
Una máquina de vectores de soporte construye un hiperplano o un conjunto de hiperplanos en un espacio dimensional alto o infinito, que se puede utilizar para clasificación, regresión u otras tareas. Intuitivamente, se logra una buena separación por el hiperplano que tiene la mayor distancia a los puntos de datos de entrenamiento más cercanos de cualquier clase (el llamado margen funcional), ya que en general cuanto mayor es el margen menor es el error de generalización del clasificador.


Las ventajas de las máquinas de vectores de soporte son:

* Efectivo en espacios de alta dimensión.
* Sigue siendo eficaz en casos en los que el número de dimensiones es mayor que el número de muestras.
* Utiliza un subconjunto de los puntos de entrenamiento en la función de decisión, por lo que es eficiente en la memoria.

In [ ]:
from sklearn.svm import SVC

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', SVC(random_state=0, kernel='linear', probability=True))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].coef_[0], X.columns)

plt.tight_layout()

In [ ]:
svc_xval_res = cross_validate_model(pipe, X, y, verbose=False)
svc_xval_res.T[['value','error']].style.format("{:.2f}")

<a id='gbrt'></a>
### Clasificador de aumento de gradiente
Gradient Tree Boosting o Gradient Boosted Regression Trees (GBRT) es una generalización del impulso a funciones de pérdida diferenciables arbitrarias. GBRT produce un modelo de predicción en forma de un conjunto de modelos de predicción débiles, típicamente árboles de decisión. Construye el modelo por etapas como lo hacen otros métodos de impulso, y los generaliza al permitir la optimización de una función de pérdida diferenciable arbitraria. GBRT es un procedimiento estándar preciso y eficaz que se puede utilizar tanto para problemas de regresión como de clasificación.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', GradientBoostingClassifier(n_estimators=250, learning_rate=0.05, random_state=0))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].feature_importances_, X.columns)

plt.tight_layout()

In [ ]:
gbc_xval_res = cross_validate_model(pipe, X, y, verbose=False)
gbc_xval_res.T[['value','error']].style.format("{:.2f}")

<a id='frt'></a>
### Bosques de árboles aleatorizados
Los árboles de decisión (DT) son un método de aprendizaje supervisado no paramétrico que se utiliza para clasificación y regresión. El objetivo es crear un modelo que prediga el valor de una variable objetivo mediante el aprendizaje de reglas de decisión simples inferidas de las características de los datos.

La técnica del bosque de árboles aleatorios incluye dos algoritmos de promediado basados ​​en árboles de decisión aleatorios: el algoritmo RandomForest y el método Extra-Trees. Ambos algoritmos son técnicas de perturbación y combinación diseñadas específicamente para árboles. Esto significa que se crea un conjunto diverso de clasificadores mediante la introducción de aleatoriedad en la construcción del clasificador. La predicción del conjunto se da como la predicción promedio de los clasificadores individuales.

<a id='rfc'></a>
#### Clasificador de bosque aleatorio

En bosques aleatorios, cada árbol del conjunto se construye a partir de una muestra extraída con reemplazo (es decir, una muestra de arranque) del conjunto de entrenamiento. Además, al dividir un nodo durante la construcción del árbol, la división que se elige ya no es la mejor división entre todas las características. En cambio, la división que se elige es la mejor división entre un subconjunto aleatorio de características. Como resultado de esta aleatoriedad, el sesgo del bosque generalmente aumenta ligeramente (con respecto al sesgo de un solo árbol no aleatorio) pero, debido al promedio, su varianza también disminuye, generalmente más que compensando el aumento del sesgo. de ahí que produzca un modelo mejor en general.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=0))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].feature_importances_, X.columns)

plt.tight_layout()

In [ ]:
rfc_xval_res = cross_validate_model(pipe, X, y, verbose=False)
rfc_xval_res.T[['value','error']].style.format("{:.2f}")

<a id='ert'></a>
#### Árboles extremadamente aleatorizados
En árboles extremadamente aleatorizados, la aleatoriedad va un paso más allá en la forma en que se calculan las divisiones. Al igual que en los bosques aleatorios, se utiliza un subconjunto aleatorio de características candidatas, pero en lugar de buscar los umbrales más discriminativos, los umbrales se dibujan al azar para cada característica candidata y el mejor de estos umbrales generados aleatoriamente se elige como la regla de división. Esto generalmente permite reducir un poco más la varianza del modelo, a expensas de un aumento ligeramente mayor del sesgo.


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

steps = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
         ('model', ExtraTreesClassifier(n_estimators=250, n_jobs=-1, random_state=0, class_weight='balanced'))]

pipe = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
pipe.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
plot_roc(pipe, X_test ,y_test)

plt.subplot(222)
plot_confusion_matrix(pipe, X_test ,y_test, normalize=True)

plt.subplot(223)
plot_proba(pipe, X_test, y_test)

plt.subplot(224)
feature_importance(pipe.named_steps['model'].feature_importances_, X.columns)

plt.tight_layout()

In [ ]:
ert_xval_res = cross_validate_model(pipe, X, y, verbose=False)
ert_xval_res.T[['value','error']].style.format("{:.2f}")

<a id='conclusion'></a>
## Comparación de modelos y conclusiones

La siguiente tabla resume el desempeño de los modelos de clasificación que consideré en este estudio. Las actuaciones se ordenan aumentando el valor de $ F_ {1} $. Los mejores rendimientos se obtienen mediante el ** árbol extremadamente aleatorio **, seguido del ** bosque aleatorio ** y la ** regresión logística **.

El árbol extremadamente aleatorizado permite identificar hasta el 66% de los préstamos que causarían un INCUMPLIMIENTO mientras se retiene el 91% de los préstamos que serían PAGADOS a tiempo. El valor de ROC AUC es tan alto como 96%, lo que indica que la probabilidad de que el clasificador se desempeñe mejor por elección aleatoria es tan baja como 4%.

In [ ]:
from collections import OrderedDict

res_comp = OrderedDict([
    ('Regresión logística'              , logit_xval_res[1:]),
    ('Clasificador SGD'                   , sgd_xval_res[1:]  ),
    ('Clasificador de vectores de apoyo'     , svc_xval_res[1:]  ),
    ('Clasificador de bosque aleatorio'         , rfc_xval_res[1:]  ),
    ('Clasificador de árboles extremadamente aleatorio' , ert_xval_res[1:]  ),
    ('Clasificador de aumento de gradiente'        , gbc_xval_res[1:]  ),
])

new_columns = {'level_0' : 'Model'}

pd.concat(res_comp).reset_index().drop('level_1', axis=1).rename(columns=new_columns).set_index('Model').sort_values('F1', ascending=False).style.format("{:.2f}")